### Setup
- To use tsne on GPU (rapidAI)
- Wandb tool

In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

***********************************************************************
Let us check on those pyarrow and cffi versions...
***********************************************************************

You're don't have pyarrow.
unloaded cffi 1.14.1
loaded cffi 1.11.5


In [ ]:
!pip install wandb -q

In [ ]:
import wandb
# WandB – Login to your wandb account so you can log all your metrics
wandb.login()

True

In [ ]:
wandb.init(project="acai-mnist-tf")

W&B Run: https://app.wandb.ai/baohq/acai-mnist-tf/runs/26hax65u

### Code

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Reshape
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np
import tqdm
import os

#### ACAI

In [ ]:
class ACAI():
    def __init__(self, img_shape=(28,28), latent_dim=32, disc_reg_coef=0.2, ae_reg_coef=0.5, dropout=0.0):
        self.latent_dim = latent_dim
        self.ae_optim = Adam(0.0001)
        self.d_optim = Adam(0.0001)
        self.img_shape = img_shape
        self.dropout = dropout
        self.disc_reg_coef = disc_reg_coef
        self.ae_reg_coef = ae_reg_coef
        self.intitializer = tf.keras.initializers.VarianceScaling(
                            scale=0.2, mode='fan_in', distribution='truncated_normal')
        self.initialize_models(self.img_shape, self.latent_dim)

    def initialize_models(self, img_shape, latent_dim):
        self.encoder = self.build_encoder(img_shape, latent_dim)
        self.decoder = self.build_decoder(latent_dim, img_shape)
        self.discriminator = self.build_discriminator(latent_dim, img_shape)
        
        img = Input(shape=img_shape)
        latent = self.encoder(img)
        res_img = self.decoder(latent)
        
        self.autoencoder = Model(img, res_img)
        discri_out = self.discriminator(img)


    def build_encoder(self, img_shape, latent_dim):
        encoder = Sequential(name='encoder')
        encoder.add(Flatten(input_shape=img_shape))
        encoder.add(Dense(1000, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        encoder.add(Dropout(self.dropout))
        encoder.add(Dense(1000, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        encoder.add(Dropout(self.dropout))
        encoder.add(Dense(latent_dim))
        
        encoder.summary()
        return encoder
    
    def build_decoder(self, latent_dim, img_shape):
        decoder = Sequential(name='decoder')
        decoder.add(Dense(1000, input_dim=latent_dim, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        decoder.add(Dropout(self.dropout))
        decoder.add(Dense(1000, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        decoder.add(Dropout(self.dropout))
        decoder.add(Dense(np.prod(img_shape), activation='sigmoid'))
        decoder.add(Reshape(img_shape))
        
        decoder.summary()
        return decoder

    def build_discriminator(self, latent_dim, img_shape):
        discriminator = Sequential(name='discriminator')
        discriminator.add(Flatten(input_shape=img_shape))
        discriminator.add(Dense(1000, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        discriminator.add(Dropout(self.dropout))
        discriminator.add(Dense(1000, activation=tf.nn.leaky_relu, kernel_initializer=self.intitializer))
        discriminator.add(Dropout(self.dropout))
        discriminator.add(Dense(latent_dim))

        # discriminator.add(Reshape((-1,)))
        discriminator.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
        
        discriminator.summary()
        return discriminator

#### Utils functions

In [ ]:
import io
from PIL import Image
from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd

def make_image_grid(imgs, shape, prefix, save_path, is_show=False):
    fig = plt.figure(figsize=[20,20])
    for index, img in enumerate(imgs):
        img = img.reshape(shape)
        ax = fig.add_subplot(10, 10, index + 1)
        ax.set_axis_off()
        ax.imshow(img, cmap='gray')
    fig.savefig(os.path.join(save_path, prefix + '.png'))
    if is_show:
        plt.show()
    plt.close(fig)
    img = Image.open(os.path.join(save_path, prefix + '.png'))

    return img

def flip_tensor(t):
    a, b = tf.split(x, 2, axis=0)
    return tf.concat([b, a], axis=0)

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

from cuml.manifold import TSNE
def visualize_latent_space(x, labels, n_clusters, range_lim=(-80, 80), perplexity=40, is_save=False, save_path=None):
    tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, n_iter=1000, init='random')
    tsne_results = tsne.fit_transform(x)
    df_subset = pd.DataFrame()
    
    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]
    df_subset['Y'] =  labels
    
    n_comps = len(np.unique(labels).tolist())
    
    plt.figure(figsize=(16,10))
    sns_plot = sns.scatterplot(
        x='tsne-2d-one', y='tsne-2d-two',
        hue='Y',
        palette=sns.color_palette(n_colors=n_comps),
        data=df_subset,
        legend="full",
        alpha=0.3
    ).set(xlim=range_lim,ylim=range_lim)
    
    if is_save:
        if not os.path.exists(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path))
        save_path = save_path if save_path else ''
        plt.savefig(save_path)
        plt.close('all')
        img = Image.open(save_path)
        return img
    return None

#### Training procedure

In [ ]:
@tf.function
def train_on_batch(x, y, model: ACAI):
    # Randomzie interpolated coefficient alpha
    alpha = tf.random.uniform((x.shape[0], 1), 0, 1)
    alpha = 0.5 - tf.abs(alpha - 0.5)  # Make interval [0, 0.5]

    with tf.GradientTape() as ae_tape, tf.GradientTape() as d_tape:
        # Constructs non-interpolated latent space and decoded input
        latent = model.encoder(x, training=True)
        res_x = model.decoder(latent, training=True)

        ae_loss = tf.reduce_mean(tf.losses.mean_squared_error(tf.reshape(x, (x.shape[0], -1)), tf.reshape(res_x, (res_x.shape[0], -1))))

        inp_latent = alpha * latent + (1 - alpha) * latent[::-1]
        res_x_hat = model.decoder(inp_latent, training=False)

        pred_alpha = model.discriminator(res_x_hat, training=True)
        # pred_alpha = K.mean(pred_alpha, [1,2,3])
        temp = model.discriminator(res_x + model.disc_reg_coef * (x - res_x), training=True)
        # temp = K.mean(temp, [1,2,3])
        disc_loss_term_1 = tf.reduce_mean(tf.square(pred_alpha - alpha))
        disc_loss_term_2 = tf.reduce_mean(tf.square(temp))

        reg_ae_loss = model.ae_reg_coef * tf.reduce_mean(tf.square(pred_alpha))

        total_ae_loss = ae_loss + reg_ae_loss
        total_d_loss = disc_loss_term_1 + disc_loss_term_2

    grad_ae = ae_tape.gradient(total_ae_loss, model.autoencoder.trainable_variables)
    grad_d = d_tape.gradient(total_d_loss, model.discriminator.trainable_variables)

    model.ae_optim.apply_gradients(zip(grad_ae, model.autoencoder.trainable_variables))
    model.d_optim.apply_gradients(zip(grad_d, model.discriminator.trainable_variables))

    return {
        'res_ae_loss': ae_loss,
        'reg_ae_loss': reg_ae_loss,
        'disc_loss': disc_loss_term_1,
        'reg_disc_loss': disc_loss_term_2

    }

In [ ]:
from collections import defaultdict
def avg_losses(total_losses):
    flatten_losses = defaultdict(list)
    for loss in total_losses:
        for kv in loss.items():
            flatten_losses[kv[0]].append(kv[1])

    avg_loss = {kv[0]: sum(kv[1]) / len(kv[1]) for kv in flatten_losses.items()}

    return avg_loss

In [ ]:
def train(model: ACAI, x_train, y_train, x_test,
          batch_size, epochs=1000, save_interval=200,
          save_path='./images'):
    n_epochs = tqdm.tqdm_notebook(range(epochs))
    total_batches = x_train.shape[0] // batch_size
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for epoch in n_epochs:
        offset = 0
        losses = []
        random_idx = np.random.randint(0, x_train.shape[0], x_train.shape[0])
        for batch_iter in range(total_batches):
            # Randomly choose each half batch
            imgs = x_train[offset:offset + batch_size,::] if (batch_iter < (total_batches - 1)) else x_train[offset:,::]
            offset += batch_size

            loss = train_on_batch(imgs, None, model)
            losses.append(loss)

        avg_loss = avg_losses(losses)
        wandb.log({'losses': avg_loss})
            
        if epoch % save_interval == 0 or (epoch == epochs - 1):
            sampled_imgs = model.autoencoder(x_test[:100])
            res_img = make_image_grid(sampled_imgs.numpy(), (28,28), str(epoch), save_path)
            
            latent = model.encoder(x_train, training=False).numpy()
            latent_space_img = visualize_latent_space(latent, y_train, 10, is_save=True, save_path=f'./latent_space/{epoch}.png')
            wandb.log({'res_test_img': [wandb.Image(res_img, caption="Reconstructed images")],
                        'latent_space': [wandb.Image(latent_space_img, caption="Latent space")]})

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255.
x_test = x_test.astype(np.float32) / 255.
ann = ACAI(dropout=0.5)
train(model=ann,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        batch_size=x_train.shape[0]//4,
        epochs=2000,
        save_interval=50,
        save_path='./images')

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                32032     
Total params: 1,818,032
Trainable params: 1,818,032
Non-trainable params: 0
_________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
# ann.autoencoder.save('acai/autoencoder')
ann.discriminator.save('/content/wandb/run-20200815_154923-26hax65u/acai/discriminator')
ann.encoder.save('/content/wandb/run-20200815_154923-26hax65u/acai/encoder')
ann.decoder.save('/content/wandb/run-20200815_154923-26hax65u/acai/decoder')

INFO:tensorflow:Assets written to: /content/wandb/run-20200815_154923-26hax65u/acai/discriminator/assets
INFO:tensorflow:Assets written to: /content/wandb/run-20200815_154923-26hax65u/acai/encoder/assets
INFO:tensorflow:Assets written to: /content/wandb/run-20200815_154923-26hax65u/acai/decoder/assets


In [ ]:
!zip /content/wandb/run-20200815_154923-26hax65u/acai.zip -r /content/wandb/run-20200815_154923-26hax65u/acai/

  adding: content/wandb/run-20200815_154923-26hax65u/acai/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/variables/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/variables/variables.index (deflated 46%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/saved_model.pb (deflated 89%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/decoder/assets/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/discriminator/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/discriminator/variables/ (stored 0%)
  adding: content/wandb/run-20200815_154923-26hax65u/acai/discriminator/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/wandb/run-20200815_154923-26